Data preprocessing for the global sales data in the udemy course below<br>
https://www.udemy.com/course/the-complete-power-bi-practical-course

In [9]:
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder.appName("GlobalSalesDataProcessing").getOrCreate()

# Read the CSV file
df = spark.read.csv('../input_data/udemy_ms_powerbi/factInternetSales.csv', header=True, inferSchema=True)

# Show the first few rows
df.show()

+----------+----------+----------+---------+-----------+------------+-----------------+----------------+--------------------+--------------+-------------+---------+--------------+--------------------+--------------+-------------------+----------------+-------+-------+--------+-------+----+--------+
|ProductKey| OrderDate|   DueDate| ShipDate|CustomerKey|PromotionKey|SalesTerritoryKey|SalesOrderNumber|SalesOrderLineNumber|RevisionNumber|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|     29|      6|      98|     19|  36|     100|
+----------+----------+----------+---------+-----------+------------+-----------------+----------------+--------------------+--------------+-------------+---------+--------------+--------------------+--------------+-------------------+----------------+-------+-------+--------+-------+----+--------+
|       310|29/12/2010|10/01/2011|5/01/2011|      21768|           1|                6|         SO43